In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(style='white',palette='muted')



from matplotlib.ticker import StrMethodFormatter

from sklearn import preprocessing, svm 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge #Lasso
from sklearn.ensemble import AdaBoostRegressor
import xgboost as xg
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, r2_score

In [44]:
df = pd.read_csv('C:\capstone\mongo\output.csv')

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_7532\601828809.py:1: SyntaxWarning: invalid escape sequence '\c'
  df = pd.read_csv('C:\capstone\mongo\output.csv')


## Preprocessing

In [45]:
df.columns
df2=df

In [46]:
df.dtypes

Unnamed: 0                                 int64
result.name.first                         object
result.name.last                          object
result.username                           object
result.password                           object
result.phoneNumber                        object
result.location.street                    object
result.location.city                      object
result.location.state                     object
result.location.country                   object
result.location.zip                       object
result.location.coordinates.latitude     float64
result.location.coordinates.longitude    float64
result.job                                object
result.salary                            float64
result.objectId                           object
id                                         int64
dtype: object

In [47]:
# renaming columns
df2.columns = df2.columns.str.replace('result.', '')
df2.columns = df2.columns.str.replace('.', '_')


In [48]:
# count
df2.shape[0]

1200

In [49]:
# checking for duplicates
duplicate = df2[df2.duplicated(['name_first','name_last'])]
duplicate.head()


,Unnamed: 0,name_first,name_last,username,password,phoneNumber,location_street,location_city,location_state,location_country,location_zip,location_coordinates_latitude,location_coordinates_longitude,job,salary,objectId,id
782,182,Napoleon,Parisian,Napoleon-Parisian,1RPuC6xASl8MTXd,401.300.0279 x9157,2041 McLaughlin Manor,Madison,North Carolina,Democratic Republic of the Congo,14157,22.3259,-94.8501,Corporate Integration Technician,251.0,66916acdc5d023f89ccba720,782


In [50]:
# # dropping duplicates
# df3=df2.drop_duplicates(subset=['name_first','name_last'])
# df3.shape[0]

In [51]:
#checking salary distribution
df3=df2
df3['salary'] = df3['salary'].apply(lambda x: x*100)
df3.boxplot(column =['salary'], grid = True)

<Axes: >

In [52]:
# checking for nulls
print(df3.isnull().sum())

Unnamed: 0                        0
name_first                        0
name_last                         0
username                          0
password                          0
phoneNumber                       0
location_street                   0
location_city                     0
location_state                    0
location_country                  0
location_zip                      0
location_coordinates_latitude     0
location_coordinates_longitude    0
job                               0
salary                            0
objectId                          0
id                                0
dtype: int64


In [53]:
df3.columns

Index(['Unnamed: 0', 'name_first', 'name_last', 'username', 'password',
       'phoneNumber', 'location_street', 'location_city', 'location_state',
       'location_country', 'location_zip', 'location_coordinates_latitude',
       'location_coordinates_longitude', 'job', 'salary', 'objectId', 'id'],
      dtype='object')

In [64]:
features_df=df3[['location_state','job','salary']]
features_df.head(3)

,location_state,job,salary
0,Michigan,Senior Usability Consultant,21700.0
1,Tennessee,National Response Representative,11400.0
2,Minnesota,Legacy Directives Analyst,42100.0


In [65]:
features_df.columns

Index(['location_state', 'job', 'salary'], dtype='object')

## Linear Regression

In [58]:
PARAM_VALIDATION_ESTIMATORS_TO_IGNORE = [ True]
global_skip_validation = True
partial_fit_and_fitted = True

In [66]:
y = features_df['salary']
X = features_df[['location_state','job']]
X = pd.get_dummies(X,columns=['location_state','job'])
y = pd.get_dummies(y,columns=['salary'])
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.1)

In [67]:

lr=LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)

# Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)
# print('Mean Squared Error: ', mean_squared_error(y_test,y_pred))'

print('R-sqaured ', r2_score(y_test,y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test,y_pred))


R-sqaured  -1.5558515229130156e+20
Mean Squared Error:  2.845026644314309e+19


## Logistic Regression

In [68]:
y = features_df['salary'] = features_df['salary']
X = features_df[['location_state','job']]
label_encoder=LabelEncoder()
y_encoded=label_encoder.fit_transform(y)
y_encoded=y_encoded.ravel()

X_encoded=pd.get_dummies(X,drop_first=True)
X_train, X_test, y_train, y_test = train_test_split (X_encoded, y_encoded, test_size=0.45)

lg = LogisticRegression()
lg.fit(X_train,y_train)
y_pred=lg.predict(X_test)

print('R-sqaured ', r2_score(y_test,y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test,y_pred))


C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_7532\2249049334.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y = features_df['salary'] = features_df['salary']


R-sqaured  -1.041669270116039
Mean Squared Error:  82462.57407407407


## XGBoost

In [69]:
y = features_df['salary'] = features_df['salary']
X = features_df[['location_state','job']]
label_encoder=LabelEncoder()
y_encoded=label_encoder.fit_transform(y)
y_encoded=y_encoded.ravel()

X_encoded = pd.get_dummies(X,drop_first=True)
X_train, X_test, y_train, y_test = train_test_split (X_encoded, y_encoded, test_size=0.3)

xgb = xg.XGBRegressor()
xgb.fit(X_train,y_train)
y_pred=xgb.predict(X_test)

print('R-sqaured ', r2_score(y_test,y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test,y_pred))

C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_7532\2308308414.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y = features_df['salary'] = features_df['salary']


R-sqaured  -0.03589463233947754
Mean Squared Error:  40926.654278208014


## AdaBoost

In [70]:
y = features_df['salary'] = features_df['salary']
X = features_df[['location_state','job']]
label_encoder=LabelEncoder()
y_encoded=label_encoder.fit_transform(y)
y_encoded=y_encoded.ravel()

X_encoded = pd.get_dummies(X,drop_first=True)
X_train, X_test, y_train, y_test = train_test_split (X_encoded, y_encoded, test_size=0.3)

adr = AdaBoostRegressor()
adr.fit(X_train,y_train)
y_pred=adr.predict(X_test)

print('R-sqaured ', r2_score(y_test,y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test,y_pred))

C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_7532\175889131.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y = features_df['salary'] = features_df['salary']


R-sqaured  -0.04693920717308697
Mean Squared Error:  41694.83122008267
